In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import os
import json
import gmaps
from config import (gkey)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
# Read in the weather data from csv file created in WeatherPy.ipynb
filepath = os.path.join('/','Users','joser', 'Documents', 'git', 'Trilogy', 'In-Class', 'Homework', '06-Python-APIs', 'UCF_Data_Bootcamp_Homework_6', 'WeatherPy', 'weather_data.csv')
weather_data = pd.read_csv(filepath)
print("Processing %d cities." % len(weather_data))

Processing 600 cities.


In [3]:
print("Total Cities : " , len(weather_data))
print("Heatmap:")

Total Cities :  600
Heatmap:


In [4]:
locations = weather_data[["Lat", "Lon"]]
rating    = weather_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
plt.savefig("01_HumidityHeatmap.png")

<Figure size 432x288 with 0 Axes>

In [5]:
# Find cities with:
#A max temperature lower than 80 degrees but higher than 70.
weather_data[ ((weather_data["Temp"] >= 70) & (weather_data["Temp"] <= 80)) & 
              (weather_data["Wind Speed"] <= 10) & 
              (weather_data["Cloudiness"] <= 1 )
            ]


,City ID,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed
63,4004834,25.9667,-100.0167,79.70,22,1,3.44
71,4720272,29.0697,-98.0819,73.53,19,1,9.22
528,2185786,-35.3167,173.7833,75.99,46,0,5.01


In [6]:
weather_data["Hotel Name"] = ""
weather_data["Address"] = ""
weather_data["Country"] = ""
weather_data.head(10)

,City ID,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed,Hotel Name,Address,Country
0,4736420,30.3480,-97.9850,64.49,29,1,6.91,,,
1,492821,53.0244,34.4392,18.00,97,100,8.88,,,
2,6360495,41.0457,-5.2677,47.71,87,75,19.57,,,
3,3064143,49.4308,17.2880,19.76,85,64,1.99,,,
4,2775819,48.4834,13.8096,27.00,94,75,3.18,,,
5,6360685,43.1079,-3.8906,58.66,63,20,24.16,,,
6,616488,40.5176,44.9319,32.00,93,75,2.30,,,
7,602426,63.8333,20.4833,1.40,92,90,3.44,,,
8,935085,-26.0000,31.5000,78.01,93,100,3.44,,,
9,3694569,-6.1519,-79.7142,82.54,48,25,8.46,,,


In [7]:
for index, row in weather_data.iterrows():
    # geocoordinates
    target_coordinates = f'{row["Lat"]},{row["Lon"]}'
    target_radius = 8000
    target_type = "lodging"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    #print(json.dumps(response, indent=2))
    try:
        weather_data.loc[index, "Lat"]         = response["results"][0]["geometry"]["location"]["lat"] # Update with actual hotel coordinates
        weather_data.loc[index, "Lon"]         = response["results"][0]["geometry"]["location"]["lng"] # Update with actual hotel coordinates
        weather_data.loc[index, "Hotel Name"]  = response["results"][0]["name"]
        weather_data.loc[index, "Address"]     = response["results"][0]["vicinity"]
        country_name                           = response["results"][0]["plus_code"]["compound_code"].split(",")[-1]
        #print(response["results"][0])
        weather_data.loc[index, "Country"]     = country_name
    except (KeyError, IndexError):
        print("Missing field/result for city id = %d. Dropping it from list."% row["City ID"])
        weather_data.drop(index,inplace=True)

Missing field/result for city id = 935085. Dropping it from list.
Missing field/result for city id = 7407690. Dropping it from list.
Missing field/result for city id = 2144117. Dropping it from list.
Missing field/result for city id = 5641875. Dropping it from list.
Missing field/result for city id = 678430. Dropping it from list.
Missing field/result for city id = 5881199. Dropping it from list.
Missing field/result for city id = 685672. Dropping it from list.
Missing field/result for city id = 1143838. Dropping it from list.
Missing field/result for city id = 63580. Dropping it from list.
Missing field/result for city id = 5460650. Dropping it from list.
Missing field/result for city id = 687933. Dropping it from list.
Missing field/result for city id = 5059925. Dropping it from list.
Missing field/result for city id = 682038. Dropping it from list.
Missing field/result for city id = 2138519. Dropping it from list.
Missing field/result for city id = 7046809. Dropping it from list.
Mi

In [8]:
# Create a map using city coordinates to set markers
marker_locations = weather_data[['Lat', 'Lon']]

# Create a marker_layer using the Hotel name, city and country to fill the info box
fig = gmaps.figure()

#Extract dataframe fields into array of tuples
records = weather_data[["Hotel Name", "Address", "Country"]].to_records(index=False)

markers = gmaps.marker_layer(marker_locations, info_box_content=[f"<dl><dt>Name:</dt><dd>{HotelName}</dd><dt>Address:</dt><dd>{Address}</dd><dt>Country:</dt><dd>{Country}</dd><dl>" for HotelName,Address,Country, *_ in records])
                         
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
plt.savefig("02_Suggested_Vacation_Locations.png")

<Figure size 432x288 with 0 Axes>